<a href="https://www.kaggle.com/code/alexanderhalpin/dnavigators-multi-task-classifier?scriptVersionId=155338676" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install torchinfo scanpy -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scanpy as sc
import pprint

In [3]:
# !curl -O https://covid19.cog.sanger.ac.uk/submissions/release1/haniffa21.processed.h5ad

In [4]:
data = sc.read_h5ad('/kaggle/input/covid19-cell-atlas-gene-expression/haniffa21.processed.h5ad')
data

AnnData object with n_obs × n_vars = 647366 × 24929
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [5]:
data.obs

,sample_id,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,full_clustering,initial_clustering,Resample,Collection_Day,...,Status,Smoker,Status_on_day_collection,Status_on_day_collection_summary,Days_from_onset,Site,time_after_LPS,Worst_Clinical_Status,Outcome,patient_id
covid_index,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAAACCTA-MH9179824,MH9179824,1160,1160,2172.0,79.0,3.637201,CD8.TE,CD8,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGAGTAATC-MH9179824,MH9179824,1419,1419,3947.0,54.0,1.368128,CD4.IL22,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGAGTGAGA-MH9179824,MH9179824,1986,1986,5886.0,143.0,2.429494,CD8.Naive,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGGAATCGC-MH9179824,MH9179824,1508,1508,4271.0,106.0,2.481854,CD4.Naive,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
AAACCTGAGTGTTGAA-MH9179824,MH9179824,1109,1109,2279.0,76.0,3.334796,CD8.EM,CD4,Initial,D0,...,Covid,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BGCV15_TTTGTCAAGGCGATAC-1,BGCV15_CV0176,2401,2401,8633.0,490.0,5.675895,CD4.CM,CD4,Resample,D28,...,Covid,Non-smoker,ITU_O2,Severe,31,Cambridge,nan,Critical,unknown,CV0176
BGCV15_TTTGTCACAAGCCATT-1,BGCV15_CV0944,1973,1973,5315.0,103.0,1.937912,NK_16hi,NK_16hi,Initial,D0,...,Healthy,Not_known,Healthy,Healthy,Healthy,Cambridge,nan,Healthy,Home,CV0944
BGCV15_TTTGTCACAGACACTT-1,BGCV15_CV0257,1209,1209,3345.0,230.0,6.875935,CD8.EM,CD8,Resample,D28,...,Covid,Non-smoker,Ward_O2,Moderate,13,Cambridge,nan,Moderate,Home,CV0257


In [6]:
data.var

,feature_types
MIR1302-2HG,Gene Expression
AL627309.1,Gene Expression
AL627309.3,Gene Expression
AL627309.2,Gene Expression
AL669831.2,Gene Expression
...,...
AB_Podocalyxin,Antibody Capture
AB_GGT1,Antibody Capture
AB_c-Met,Antibody Capture
AB_LIGHT,Antibody Capture


In [7]:
def explore_obs(data, tag):
    values, counts = np.unique(data.obs[tag], return_counts=True)
    
    if len(values) > 1000:
        return
    
    print(f"{len(values)} unique {tag} elements")
    print(f"--------------------------------------------")
    
    max_value_len = max([len(str(value)) for value in values])
    max_count_len = max([len(str(count)) for count in counts])
    
    for i, (value, count) in enumerate(reversed(sorted(list(zip(values, counts)), key = lambda x: x[1]))):
        print(f"{value:<{max_value_len+3}} {count:>{max_count_len}}")

In [8]:
import torch
import numpy as np
import torch.nn as nn
from torchinfo import summary

In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cpu'

In [26]:
class SplitActivator(nn.Module):
    def __init__(self, latent_features, mean_act = nn.Tanh(), logvar_act = nn.ReLU()):
        super().__init__()

        self.latent_features = latent_features
        self.mean_act = mean_act
        self.logvar_act = logvar_act
    
    def forward(self, x):
#         print(f"splitinx: {x.shape}")
        mean, logvar = torch.split(x, self.latent_features, dim=-1)
#         logvar = torch.clamp(logvar, min=1e-6)

        mean = self.mean_act(mean)
        logvar = self.logvar_act(logvar)
        
#         print(f"postsplit: {mean.shape, logvar.shape}")
        
        return torch.cat((mean, logvar), axis=-1)

In [27]:
class Sampler(nn.Module):

    def __init__(self, latent_features):
        super().__init__()
        self.latent_features = latent_features

    def forward(self, mean, logvar):
        std = torch.exp(logvar*0.5)

        epsilon = torch.normal(torch.zeros(mean.shape), torch.ones(std.shape)).to(device)

        sample = mean + epsilon * std

        return sample


In [28]:
class VarAutoencoder(nn.Module):

    def __init__(self, input_features, latent_features, classifier_dict):
        super().__init__()
        
        self.classifier_dict = classifier_dict

        self.input_features = input_features
        self.latent_features = latent_features
        self.hidden_features = 2*latent_features
        self.output_features = sum((len(e) for e in classifier_dict.values()))
        
        self.encoder = nn.Sequential(
#             nn.Sigmoid(),
            nn.Linear(self.input_features, self.hidden_features),
            nn.ReLU(),
            nn.Linear(self.hidden_features, self.hidden_features),
            nn.ReLU(),
            nn.Linear(self.hidden_features, 2*self.latent_features),
#             SplitActivator(self.latent_features),
        )

        self.sampler = Sampler(self.latent_features)

        self.decoder = nn.Sequential(
            nn.Linear(self.latent_features, self.hidden_features),
            nn.ReLU(),
            nn.Linear(self.hidden_features, self.output_features),
#             nn.Sigmoid(),
        )
        
    def encode(self, x, split=True):
        
        x = self.encoder(x)
        
        if split:
            mean, logvar = torch.split(x, self.latent_features, dim=-1)
            return mean, logvar
        
        return x
        
    def embed(self, x, return_params=False):
        mean, logvar = self.encode(x, split=True)
        z = self.sampler(mean, logvar)
        
        if return_params:
            return z, mean, logvar
        
        return z

    def forward(self, x):
        
#         x = self.encoder(x)
#         mean, logvar = torch.split(x, self.latent_features, dim=-1)
# #             mean, logvar = self.encode(x, split=True)
#         z = self.sampler(mean, logvar)
#         xx = self.decoder(z)

        z, mean, logvar = self.embed(x, return_params=True)
        xx = self.decoder(z)

        return xx, mean, logvar


In [29]:
class KLDivergence(nn.Module):

    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha

    def forward(self, mean, logvar):
        kl = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
        return kl * self.alpha


In [84]:
class SCRNASeqDataset(torch.utils.data.Dataset):

    def __init__(self, data, classifier_obs):
        self.classifier_obs = sorted(classifier_obs)
        self.classifier_output_obs = [f"{ob}_class" for ob in self.classifier_obs]
        self.unique_dict = self.__get_unique_dict(data, self.classifier_obs)
        self.class_sizes = tuple(len(self.unique_dict[ob]) for ob in self.classifier_obs)
        
        for ob in self.classifier_obs:
            data.obs[f"{ob}_class"] = data.obs[ob].map(self.unique_dict[ob])
    
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        sample = torch.tensor(np.squeeze(self.data.X[idx].A).astype(np.float32))
        labels = torch.tensor(self.data.obs[self.classifier_output_obs].iloc[idx].to_numpy())
        labels = tuple(torch.squeeze(t) for t in torch.split(labels, 1, dim=-1))

        return sample, labels
    
    def __get_unique_dict(self, data, classifier_obs):
        unique_dict = {}

        for ob in classifier_obs:
            unique_dict[ob] = {}

            values = np.unique(data.obs[ob])
            for v, value in enumerate(values):
                unique_dict[ob][value] = v
        
        return unique_dict
        

In [101]:
def train_epoch(dataloader, model, classification_loss_fns, regular_loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    sample_counter = 0
    for batch, (x, ys) in enumerate(dataloader):
        optimizer.zero_grad()

        x = x.to(device)
        ys = tuple(y.to(device) for y in ys)

        # Reconstruction and regularization losses
        x = torch.cat((x, ((x>0)*1.0)), axis=-1)
        y_hat, mean, logvar = model(x)
        regular_loss = regular_loss_fn(mean, logvar)
        
        y_hats = torch.split(y_hat, dataloader.dataset.dataset.class_sizes, dim=-1)
        
        classification_loss = 0
        for c in range(len(y_hats)):
            classification_loss += classification_loss_fns[c](y_hats[c], ys[c])
        
        # Backpropagation
        loss = regular_loss + classification_loss
        loss.backward()
        optimizer.step()

        sample_counter += len(x)
        if batch % 1000 == 0:
            loss = loss.item()/len(x)
            regular_loss = regular_loss.item()/len(x)
            classification_loss = classification_loss.item()/len(x)
            
            print(f"[{batch:>5d}/{len(dataloader):>5d}] L: {loss:>7e}  KL: {regular_loss:>7e} CE: {classification_loss:>7e}")

In [102]:
def test(dataloader, model, classification_loss_fns, regular_loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()

    regular_loss = 0
    classification_loss = 0
    with torch.no_grad():
        for x,ys in dataloader:
            x = x.to(device)
            ys = tuple(y.to(device) for y in ys)
            
            # Reconstruction and regularization losses
            x = torch.cat((x, ((x>0)*1.0)), axis=-1)
            y_hat, mean, logvar = model(x)
            
            y_hats = torch.split(y_hat, dataloader.dataset.dataset.class_sizes, dim=-1)

            classification_loss = 0
            for c in range(len(y_hats)):
                classification_loss += classification_loss_fns[c](y_hats[c], ys[c])
            
            regular_loss += regular_loss_fn(mean, logvar)

    avg_regular = regular_loss / num_batches
    avg_classification = classification_loss / num_batches
    avg_total = avg_regular + avg_classification
    print(f"loss total: {(avg_total):>8e}\t regular: {(avg_regular):>8e}\t class: {(avg_classification):>8e}\n")

In [64]:
n_least_sparse_samples = 100_000
data_subset = data[data.obs['n_genes_by_counts'].argsort()[-n_least_sparse_samples:]]
data_subset

View of AnnData object with n_obs × n_vars = 100000 × 24929
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [18]:
nz_vals_per_gene = np.array((data_subset.X > 0.0).sum(axis=0))[0]

In [19]:
n_most_read_genes = 2000
data_subset_gene_subset = data_subset[:, nz_vals_per_gene.argsort()[-n_most_read_genes:]]
data_subset_gene_subset

View of AnnData object with n_obs × n_vars = 100000 × 2000
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [ ]:
data_subset_gene_subset.write_h5ad(f"haniffa21_subset_{n_least_sparse_samples}_{n_most_read_genes}.h5ad")

In [20]:
import shutil
import os

os.mkdir("/kaggle/output")
shutil.copyfile(f"haniffa21_subset_{n_least_sparse_samples}_{n_most_read_genes}.h5ad", f"/kaggle/output/haniffa21_subset_{n_least_sparse_samples}_{n_most_read_genes}.h5ad")

FileNotFoundError: [Errno 2] No such file or directory: 'haniffa21_subset_100000_2000.h5ad'

In [77]:
classifier_obs = ("initial_clustering", "Status", "Status_on_day_collection_summary")

In [92]:
gex_dataset = SCRNASeqDataset(data_subset_gene_subset, classifier_obs)
train_set, val_set = torch.utils.data.random_split(gex_dataset, [0.85, 0.15])

gex_train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True)
gex_val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=16, shuffle=True)

In [93]:
gex_dataset.classifier_obs

['Status', 'Status_on_day_collection_summary', 'initial_clustering']

In [94]:
gex_dataset.unique_dict

{'Status': {'Covid': 0, 'Healthy': 1, 'LPS': 2, 'Non_covid': 3},
 'Status_on_day_collection_summary': {'Asymptomatic': 0,
  'Critical': 1,
  'Healthy': 2,
  'LPS_10hours': 3,
  'LPS_90mins': 4,
  'Mild': 5,
  'Moderate': 6,
  'Non_covid': 7,
  'Severe': 8},
 'initial_clustering': {'B_cell': 0,
  'CD14': 1,
  'CD16': 2,
  'CD4': 3,
  'CD8': 4,
  'DCs': 5,
  'HSC': 6,
  'Lymph_prolif': 7,
  'MAIT': 8,
  'Mono_prolif': 9,
  'NK_16hi': 10,
  'NK_56hi': 11,
  'Plasmablast': 12,
  'Platelets': 13,
  'RBC': 14,
  'Treg': 15,
  'gdT': 16,
  'pDC': 17}}

In [95]:
gex_dataset.class_sizes

(4, 9, 18)

In [81]:
model = VarAutoencoder(2*n_most_read_genes, 128, gex_dataset.unique_dict)
model = model.to(device)
summary(model, input_size=(69, 2*n_most_read_genes))

Layer (type:depth-idx)                   Output Shape              Param #
VarAutoencoder                           [69, 31]                  --
├─Sequential: 1-1                        [69, 256]                 --
│    └─Linear: 2-1                       [69, 256]                 1,024,256
│    └─ReLU: 2-2                         [69, 256]                 --
│    └─Linear: 2-3                       [69, 256]                 65,792
│    └─ReLU: 2-4                         [69, 256]                 --
│    └─Linear: 2-5                       [69, 256]                 65,792
├─Sampler: 1-2                           [69, 128]                 --
├─Sequential: 1-3                        [69, 31]                  --
│    └─Linear: 2-6                       [69, 256]                 33,024
│    └─ReLU: 2-7                         [69, 256]                 --
│    └─Linear: 2-8                       [69, 31]                  7,967
Total params: 1,196,831
Trainable params: 1,196,831
Non-trainab

In [105]:
regular_loss_fn = KLDivergence(0.1)
classification_loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1E-2)

In [72]:
import gc
gc.collect()

2460

In [107]:
num_epochs = 10

classification_loss_fns = tuple(nn.CrossEntropyLoss() for _ in range(len(gex_dataset.classifier_obs)))
for e in range(num_epochs):
    print(f"Epoch {e+1}\n-------------------------------")
    train_epoch(gex_train_dataloader, model, classification_loss_fns, regular_loss_fn, optimizer)
    test(gex_val_dataloader, model, classification_loss_fns, regular_loss_fn)

Epoch 1
-------------------------------
[    0/ 5313] L: 3.462478e-01  KL: 6.871581e-04 CE: 3.455607e-01
[ 1000/ 5313] L: 3.111422e-01  KL: 5.765021e-04 CE: 3.105657e-01
[ 2000/ 5313] L: 2.938735e-01  KL: 2.733573e-03 CE: 2.911399e-01
[ 3000/ 5313] L: 2.623125e-01  KL: 3.757197e-03 CE: 2.585553e-01
[ 4000/ 5313] L: 2.516191e-01  KL: 3.003806e-04 CE: 2.513187e-01
[ 5000/ 5313] L: 2.811180e-01  KL: 4.918426e-04 CE: 2.806262e-01
loss total: 1.107731e-02	 regular: 7.090091e-03	 class: 3.987214e-03

Epoch 2
-------------------------------
[    0/ 5313] L: 3.412878e-01  KL: 4.433632e-04 CE: 3.408445e-01
[ 1000/ 5313] L: 3.287057e-01  KL: 4.710168e-04 CE: 3.282347e-01
[ 2000/ 5313] L: 2.825827e-01  KL: 4.148841e-04 CE: 2.821678e-01
[ 3000/ 5313] L: 2.899895e-01  KL: 5.265772e-04 CE: 2.894629e-01
[ 4000/ 5313] L: 2.877931e-01  KL: 4.973352e-04 CE: 2.872958e-01
[ 5000/ 5313] L: 3.233457e-01  KL: 1.297703e-03 CE: 3.220480e-01
loss total: 1.739160e-02	 regular: 1.318009e-02	 class: 4.211507e-03



KeyboardInterrupt: 

In [42]:
tuple(len(ob_dict) for ob_dict in gex_dataset.unique_dict.values())

(18, 4, 9)

In [100]:
x,ys = gex_dataset[0]

x = torch.cat((x, ((x>0)*1.0)), axis=-1)

xx, mean, logvar = model(x.to(device))

print(ys)
print(xx)

(tensor(1), tensor(2), tensor(0))
tensor([ 0.0481, -0.0821,  0.4026,  0.1544, -0.3169,  0.4199, -0.1695, -0.0341,
        -0.3337,  0.1795, -0.5660,  0.0990, -0.1936,  0.1541,  0.0117, -0.3608,
         0.0376, -0.0963,  0.1796, -0.1222,  0.3308, -0.5518,  0.2896,  0.0572,
        -0.2054, -0.1971, -0.0722,  0.3280, -0.2590,  0.3859, -0.1359],
       grad_fn=<AddBackward0>)


In [49]:
(x > 0)*1.0

tensor([0., 0., 0.,  ..., 1., 1., 1.])

In [119]:
np.random.seed(42)
shuffled_indices = np.arange(len(gex_dataset))
np.random.shuffle(shuffled_indices)

In [123]:
n_tsne = 1000
x = gex_dataset[shuffled_indices[:n_tsne]][0]
x = torch.cat((x, ((x>0)*1.0)), axis=-1)
y = gex_dataset.data[shuffled_indices[:n_tsne]].obs['Status']

In [124]:
tsne_sample = model.embed(x.to(device)).clamp(min=1E-9, max=1.0).cpu().detach().numpy()

In [126]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(tsne_sample)
tsne.kl_divergence_

2.719663143157959

In [127]:
fig = px.scatter(x=X_tsne[:, 0], y=X_tsne[:, 1], color=y)
fig.update_layout(
    title="t-SNE visualization of Custom Classification dataset",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()

In [ ]:
#placeholder random data
atac_dataset = SCRNASeqDataset(gex_feature_filtered.X)

atac_train_dataloader = DataLoader(atac_dataset[:50_000], batch_size=16, shuffle=True)
atac_val_dataloader = DataLoader(atac_dataset[50_000:], batch_size=16, shuffle=True)

In [ ]:
atac_dataset.data.shape

In [ ]:
model2 = VarAutoencoder(n_best_features, 32)
model2 = model2.to(device)
summary(model2, input_size=(69, n_best_features))

In [ ]:
regular_loss_fn2 = KLDivergence(0.1)
recon_loss_fn2 = MSELoss(1.0)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=5E-3)

In [ ]:
num_epochs = 30

for e in range(num_epochs):
    print(f"Epoch {e+1}\n-------------------------------")
    train_epoch(atac_train_dataloader, model2, recon_loss_fn2, regular_loss_fn2, optimizer2)
    test(atac_val_dataloader, model2, regular_loss_fn2, recon_loss_fn2)

In [ ]:
tsne_subset2 = atac_feature_filtered[shuffled_indices[:1000], :]
y2 = tsne_subset2.obs['cell_type']

In [ ]:
tsne_sample2 = model2.embed(torch.tensor(tsne_subset2.X.A).to(device)).clamp(min=1E-9, max=1.0).cpu().detach().numpy()

In [ ]:
# from sklearn.manifold import TSNE
import plotly.express as px

tsne2 = TSNE(n_components=2, random_state=42)
X_tsne2 = tsne2.fit_transform(tsne_sample)
tsne2.kl_divergence_

In [ ]:
fig2 = px.scatter(x=X_tsne2[:, 0], y=X_tsne2[:, 1], color=y2)
fig2.update_layout(
    title="t-SNE visualization of Custom Classification dataset",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig2.show()